## Deep Learning Group Project

Notebook done by:

- André Sousa 20240517
- Francisco Pontes 20211583
- Isabella Costa 20240685
- Jéssica Cristas 20240488
- Tiago Castilho 20240489

________


## Models

________


### Google Drive Mounting and Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
src_path = "/content/drive/MyDrive/Deep Learning/Group17_DL_Project/utils.py"

dst_path = "/content/utils.py"

!cp "{src_path}" "{dst_path}"


In [31]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from keras.metrics import Accuracy, F1Score
from tensorflow.keras.utils import image_dataset_from_directory
from utils import *
import matplotlib.pyplot as plt


In [9]:
#split_data('/content/drive/MyDrive/Deep Learning/Group17_DL_Project/new_rare_species_filtered','/content/drive/MyDrive/Deep Learning/Group17_DL_Project')


### HyperParameters and Constants

In [23]:
n_classes = 202
batch_size = 32
#epochs = 5
input_shape = (299, 299, 3)
#image_size = (299, 299)
optimizer = Adam(learning_rate=1e-4, name="optimizer")
loss = CategoricalCrossentropy(from_logits=False, name="categorical_crossentropy_loss")
AUTOTUNE = tf.data.AUTOTUNE

In [24]:
os.makedirs("/content/drive/MyDrive/Deep Learning/Group17_DL_Project/tfrecords", exist_ok=True)
paths = {
    "train": "/content/drive/MyDrive/Deep Learning/Group17_DL_Project/tfrecords/train.tfrecord",
    "val": "/content/drive/MyDrive/Deep Learning/Group17_DL_Project/tfrecords/val.tfrecord",
    "test": "/content/drive/MyDrive/Deep Learning/Group17_DL_Project/tfrecords/test.tfrecord"
}

In [25]:
train_dir_path = "/content/drive/MyDrive/Deep Learning/Group17_DL_Project/train"
val_dir_path = "/content/drive/MyDrive/Deep Learning/Group17_DL_Project/val"
test_dir_path = "/content/drive/MyDrive/Deep Learning/Group17_DL_Project/test"
metadata_path = "/content/drive/MyDrive/Deep Learning/Group17_DL_Project/metadata.csv"

### Importing Data and Transforming to Tensor Flow Records

In [13]:
base_train_ds = image_dataset_from_directory(train_dir_path,
    batch_size=batch_size,
    verbose=False
)

base_val_ds = image_dataset_from_directory(
    val_dir_path,
    batch_size=batch_size,
    verbose=False
)

base_test_ds = image_dataset_from_directory(
    test_dir_path,
    batch_size=batch_size,
    verbose=False
)

In [14]:
# export tf records
if not os.path.exists(paths["train"]):
    write_tfrecord(base_train_ds, paths["train"], apply_augmentation=True)

if not os.path.exists(paths["val"]):
    write_tfrecord(base_val_ds, paths["val"])

if not os.path.exists(paths["test"]):
    write_tfrecord(base_test_ds, paths["test"])

In [15]:
def parse_record(example_proto):
    features = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64)
    }
    parsed = tf.io.parse_single_example(example_proto, features)
    image = tf.io.decode_jpeg(parsed['image'], channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_with_pad(image, 299, 299)
    return image, parsed['label']

def one_hot_label(x, y):
    return x, tf.one_hot(y, depth=202)

In [16]:
train_ds = tf.data.TFRecordDataset(paths["train"])
train_ds = train_ds.map(parse_record, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(one_hot_label).shuffle(1000).batch(batch_size).prefetch(AUTOTUNE)

val_ds = tf.data.TFRecordDataset(paths["val"])
val_ds = val_ds.map(parse_record, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(one_hot_label).batch(batch_size).prefetch(AUTOTUNE)

test_ds = tf.data.TFRecordDataset(paths["test"])
test_ds = test_ds.map(parse_record, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(one_hot_label).batch(batch_size).prefetch(AUTOTUNE)


# Visualize Data Augmentation

________


In [27]:
data_augmentation = data_aug()
normalization_layer = layers.Rescaling(1./255)

In [28]:
data_augmentation.build(input_shape= (299, 299, 3))
data_augmentation.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip_1 (RandomFlip)      │ (299, 299, 3)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_1               │ (299, 299, 3)          │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_1 (RandomZoom)      │ (299, 299, 3)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_contrast_1               │ (299, 299, 3)          │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_brightness_1             │ (299, 299, 3)          │             0 │
│ (RandomBrightness)              │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [29]:
def visualize_augmentations(generator, num_augments=3):
    """Displays original images and their augmented versions from a dataset generator."""

    # Set up the figure with enough room
    plt.figure(figsize=(15, 10))

    # Take just one batch from the dataset
    for images, labels in generator.take(1):
        for row_index, image in enumerate(images[:3]):  # Limit to 3 original images
            label_index = np.argmax(labels[row_index])
            class_name = generator.class_names[label_index]

            # --- Show Original Image ---
            ax = plt.subplot(3, num_augments + 1, row_index * (num_augments + 1) + 1)
            plt.imshow(image.numpy().astype("uint8"))
            plt.title(f"Original ({image.shape}) - {class_name}")
            plt.axis("off")

            # --- Show Augmented Versions ---
            for j in range(num_augments):
                augmented_image = data_augmentation(image[None, ...], training=True)[0]
                ax = plt.subplot(3, num_augments + 1, row_index * (num_augments + 1) + 2 + j)
                plt.imshow(augmented_image.numpy().astype("uint8"))
                plt.title(f"Augmented #{j + 1}")
                plt.axis("off")

    plt.tight_layout()
    plt.show()

In [32]:
visualize_augmentations(base_train_ds)

Output hidden; open in https://colab.research.google.com to view.

### Class Weights

In [17]:
labels_list = []

for _, labels in train_ds.unbatch():
    class_idx = tf.argmax(labels).numpy()
    labels_list.append(class_idx)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(n_classes),
    y=labels_list
)

class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

### CallBacks

In [18]:
callbacks = [
        EarlyStopping(
            monitor='val_loss',
            mode='min',
            patience=5,
            restore_best_weights=True,
            min_delta=0.1
        ),
        ModelCheckpoint(
            filepath=f'model_from_scratch.keras',
            monitor='val_macro_f1',
            mode='max',
            save_best_only=True,
            save_weights_only=False
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            verbose=1,
            min_lr=1e-7
        )
    ]

________


### Model Training

In [19]:
metrics=['accuracy', F1Score(average='macro',)]

In [20]:
#optuna results with some finetunning
learning_rate =  0.00022631443241333675
dropout = 0.5
dense_units = 256
trainable_layers = 40

In [21]:
def simple_cnn(input_shape):
    simple_cnn = tf.keras.Sequential([
        layers.Input(shape = input_shape),
        layers.Rescaling(1./255),

        layers.Conv2D(filters=3, kernel_size=5, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(pool_size = 2),

        layers.Conv2D(filters=3, kernel_size=5, padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(pool_size = 2),

        layers.Flatten(),
        layers.Dropout(0.4),
        layers.Dense(units=202, activation="softmax")])
    return simple_cnn


model = simple_cnn((299, 299, 3))
model.compile(optimizer=Adam(learning_rate=learning_rate), loss=loss, metrics=metrics)
model.summary()

hist_f1_test = model.fit(
    train_ds,
        validation_data=val_ds,
        epochs=50,
        class_weight=class_weights_dict,
        callbacks=callbacks,
        verbose=2)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 299, 299, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 299, 299, 3)    │           228 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 299, 299, 3)    │            12 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 299, 299, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 149, 149, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 149, 149, 3)    │           228 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 149, 149, 3)    │            12 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 149, 149, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 74, 74, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16428)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16428)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 202)            │     3,318,658 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,319,138 (12.66 MB)

 Trainable params: 3,319,126 (12.66 MB)

 Non-trainable params: 12 (48.00 B)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


240/240 - 17s - 72ms/step - accuracy: 0.0104 - f1_score: 0.0056 - loss: 5.4126 - val_accuracy: 0.0030 - val_f1_score: 3.0003e-05 - val_loss: 5.3100 - learning_rate: 2.2631e-04
Epoch 2/50


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with val_macro_f1 available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


240/240 - 4s - 18ms/step - accuracy: 0.0720 - f1_score: 0.0681 - loss: 4.6826 - val_accuracy: 0.0225 - val_f1_score: 0.0081 - val_loss: 5.2674 - learning_rate: 2.2631e-04
Epoch 3/50
240/240 - 4s - 18ms/step - accuracy: 0.3772 - f1_score: 0.4342 - loss: 2.4399 - val_accuracy: 0.0030 - val_f1_score: 3.0021e-05 - val_loss: 44.2714 - learning_rate: 2.2631e-04
Epoch 4/50
240/240 - 4s - 18ms/step - accuracy: 0.6914 - f1_score: 0.7599 - loss: 0.9604 - val_accuracy: 0.0255 - val_f1_score: 5.9752e-04 - val_loss: 58.3308 - learning_rate: 2.2631e-04
Epoch 5/50

Epoch 5: ReduceLROnPlateau reducing learning rate to 0.000113157213490922.
240/240 - 4s - 18ms/step - accuracy: 0.8273 - f1_score: 0.8797 - loss: 0.4613 - val_accuracy: 0.0310 - val_f1_score: 8.1537e-04 - val_loss: 57.7986 - learning_rate: 2.2631e-04
Epoch 6/50
240/240 - 4s - 18ms/step - accuracy: 0.9067 - f1_score: 0.9394 - loss: 0.2475 - val_accuracy: 0.0280 - val_f1_score: 0.0010 - val_loss: 40.0076 - learning_rate: 1.1316e-04


### Evaluating and saving the model

In [22]:
final_model = model
test_results = final_model.evaluate(test_ds, verbose=2)
test_metrics = dict(zip(final_model.metrics_names, test_results))

print("\ntest_ds", test_metrics)

final_model.save(f'/content/drive/MyDrive/Deep Learning/model_from_scratch_final.keras')


52/52 - 3s - 54ms/step - accuracy: 0.0024 - f1_score: 2.4002e-05 - loss: 5.3101

test_ds {'loss': 5.310118675231934, 'compile_metrics': 0.002430133754387498}
